In [95]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [96]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [97]:
train = pd.read_csv('../data/prep/train3.csv')
test = pd.read_csv('../data/prep/test3.csv')
submission = pd.read_csv('../data/raw/housing/sample_submission.csv')

In [98]:
train

,id,Overall.Qual,Gr.Liv.Area,Exter.Qual,Garage.Cars,Garage.Area,Kitchen.Qual,Total.Bsmt.SF,X1st.Flr.SF,Bsmt.Qual,...,Year.Built,Year.Remod.Add,Garage.Yr.Blt,Garage.Area.Cars,comb.Qual,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,target
0,1,2.397895,7.780303,1.791759,1.386294,6.876265,1.791759,7.780303,7.780303,1.791759,...,7.602900,7.602900,7.602900,5.779714,3.258097,2.197225,2.197225,2.197225,3.218876,12.864243
1,2,2.079442,7.210080,1.609438,1.098612,6.146329,1.609438,7.210080,7.210080,1.791759,...,7.604396,7.604894,7.604396,5.455321,3.044522,1.791759,1.609438,1.791759,2.708050,12.175619
2,3,1.791759,6.803505,1.386294,0.693147,5.666427,1.386294,6.762730,6.803505,1.386294,...,7.584773,7.584773,7.584773,5.666427,2.708050,3.806662,3.806662,3.806662,4.890349,11.719948
3,4,1.791759,7.069023,1.386294,1.098612,6.357842,1.609438,6.523562,6.523562,1.386294,...,7.550135,7.604396,7.601402,5.666427,2.772589,2.484907,1.791759,4.718499,4.852030,11.813037
4,5,2.079442,7.580189,1.609438,1.386294,6.842683,1.609438,6.934397,6.934397,1.609438,...,7.603898,7.603898,7.603898,5.746203,2.995732,1.945910,1.945910,1.945910,2.944439,12.429220
5,6,2.197225,7.585281,1.609438,1.386294,6.523562,1.791759,6.652863,6.652863,1.791759,...,7.605890,7.606387,7.605890,5.427883,3.135494,1.098612,0.693147,1.098612,1.791759,12.504327
6,7,1.945910,7.299121,1.386294,1.098612,6.093570,1.386294,7.299121,7.299121,1.386294,...,7.579679,7.579679,7.579679,5.402677,2.772589,4.007333,4.007333,4.007333,5.093750,11.960818
7,8,2.197225,7.833996,1.609438,1.098612,6.297109,1.609438,7.833996,7.833996,1.609438,...,7.591862,7.591862,7.591862,5.605802,3.044522,3.433987,3.433987,3.433987,4.510860,12.535380
8,9,2.197225,7.882315,1.609438,1.386294,6.616065,1.609438,7.299797,7.323831,1.791759,...,7.601902,7.602401,7.601902,5.520127,3.091042,2.397895,2.302585,2.397895,3.401197,12.950984
9,10,2.197225,7.273093,1.609438,1.098612,6.148468,1.609438,7.267525,7.273093,1.791759,...,7.602900,7.602900,7.602900,5.457456,3.091042,2.197225,2.197225,2.197225,3.218876,12.356650


In [99]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [100]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [101]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [102]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [103]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [104]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [105]:
rf_pred = np.zeros(target.shape[0])
rf_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rf_val)} & std = {np.std(rf_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.0844100683928693

2 FOLD Training.....
2 FOLD NMAE = 0.10469761757467397

3 FOLD Training.....
3 FOLD NMAE = 0.10159300024833884

4 FOLD Training.....
4 FOLD NMAE = 0.12284490789779805

5 FOLD Training.....
5 FOLD NMAE = 0.09617324196958335

6 FOLD Training.....
6 FOLD NMAE = 0.09499782569598449

7 FOLD Training.....
7 FOLD NMAE = 0.09059950443650648

8 FOLD Training.....
8 FOLD NMAE = 0.10421959191205075

9 FOLD Training.....
9 FOLD NMAE = 0.10425503573549091

10 FOLD Training.....
10 FOLD NMAE = 0.10485432845093107

10FOLD Mean of NMAE = 0.10086451223142272 & std = 0.009846710935203833


In [106]:
gbr_pred = np.zeros(target.shape[0])
gbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    gbr = GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate = 0.05, n_estimators = 1000)
    gbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(gbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    gbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = gbr.predict(target) / 10
    gbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(gbr_val)} & std = {np.std(gbr_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.08349468604339731

2 FOLD Training.....
2 FOLD NMAE = 0.0973470372583205

3 FOLD Training.....
3 FOLD NMAE = 0.09856276426859621

4 FOLD Training.....
4 FOLD NMAE = 0.11347557449097499

5 FOLD Training.....
5 FOLD NMAE = 0.09124773837718772

6 FOLD Training.....
6 FOLD NMAE = 0.10046034871929486

7 FOLD Training.....
7 FOLD NMAE = 0.09942803743432507

8 FOLD Training.....
8 FOLD NMAE = 0.0951156067139962

9 FOLD Training.....
9 FOLD NMAE = 0.0869594714406881

10 FOLD Training.....
10 FOLD NMAE = 0.09819040126774937

10FOLD Mean of NMAE = 0.09642816660145302 & std = 0.007809410300008837


In [107]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2931341	test: 184599.8953464	best: 184599.8953464 (0)	total: 3.68ms	remaining: 11s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 184599.8379
bestIteration = 160

Shrink model to first 161 iterations.
1 FOLD NMAE = 0.07661230780059615

2 FOLD Training.....
0:	learn: 0.2937850	test: 182774.3366574	best: 182774.3366574 (0)	total: 3.98ms	remaining: 11.9s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 182774.3159
bestIteration = 87

Shrink model to first 88 iterations.
2 FOLD NMAE = 0.11513412450154048

3 FOLD Training.....
0:	learn: 0.2879067	test: 196831.3019972	best: 196831.3019972 (0)	total: 2.48ms	remaining: 7.45s
1000:	learn: 0.0671934	test: 196831.2235266	best: 196831.2205092 (491)	total: 2.17s	remaining: 4.33s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 196831.2205
bestIteration = 491

Shrink model to first 492 iterations.
3 FOLD NMAE = 0.09536267448150217

4 FOLD Training.....
0:	learn:

In [108]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.07759123821384555

2 FOLD Training.....
2 FOLD NMAE = 0.09838485905746261

3 FOLD Training.....
3 FOLD NMAE = 0.09881404545829854

4 FOLD Training.....
4 FOLD NMAE = 0.10678691078675713

5 FOLD Training.....
5 FOLD NMAE = 0.08748296290849022

6 FOLD Training.....
6 FOLD NMAE = 0.10044471313753561

7 FOLD Training.....
7 FOLD NMAE = 0.0948358937007788

8 FOLD Training.....
8 FOLD NMAE = 0.09194441948718975

9 FOLD Training.....
9 FOLD NMAE = 0.0885833690411214

10 FOLD Training.....
10 FOLD NMAE = 0.0953867328416775

10FOLD Mean of NMAE = 0.09402551446331572 & std = 0.007715835741467004


In [111]:
np.exp((ngb_pred + cb_pred + rf_pred + gbr_pred) / 4)

array([325539.64004664, 131934.43073157, 176394.31576   , ...,
        80456.51411983, 198593.63817291, 136370.33102249])

In [ ]:
submission['target'] = np.expm1((ngb_pred + cb_pred + rf_pred + gbr_pred) / 4)

In [ ]:
np.exp(ngb_pred)

In [115]:
submission['target'] = np.exp(ngb_pred)

In [116]:
submission

,id,target
0,1,337229.361199
1,2,130971.872148
2,3,177784.774309
3,4,247967.072247
4,5,133861.341429
5,6,149115.488352
6,7,191852.868232
7,8,119025.250145
8,9,138874.054617
9,10,302768.101520


In [118]:
submission.to_csv("../out/ngb/ngb1.csv", header = True, index = False)